In [ ]:
# ============================================================================
# B02: Working with Phase
# ============================================================================
# This notebook explores phase as a circular (angular) variable and builds
# the foundation for phase-based connectivity metrics like PLV.
#
# Prerequisites: B01 (The Hilbert Transform)
# ============================================================================

# Standard library imports
import numpy as np
from numpy.typing import NDArray
from typing import Optional, Tuple
import matplotlib.pyplot as plt
from scipy.signal import hilbert
import sys
from pathlib import Path

# Add src to path for local imports
src_path = Path.cwd().parents[2] / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Local imports
from colors import COLORS
from filtering import bandpass_filter
from hilbert import compute_instantaneous_phase, compute_envelope

# Extract color aliases for convenience
PRIMARY_BLUE = COLORS["signal_1"]      # Sky Blue
PRIMARY_RED = COLORS["negative"]        # Coral Red
PRIMARY_GREEN = COLORS["signal_3"]      # Sage Green
ACCENT_PURPLE = COLORS["signal_5"]      # Lavender
ACCENT_TEAL = COLORS["signal_6"]        # Soft Teal
ACCENT_GOLD = COLORS["signal_4"]        # Golden
SECONDARY_PINK = COLORS["signal_2"]     # Rose Pink

# Plot style
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("B02: Working with Phase")
print("=" * 50)
print("Understanding phase as a circular variable")
print("Building tools for phase-based connectivity analysis")

---

## 1. Introduction

Phase synchronization is one of the most important mechanisms for neural communication. The **"Communication through Coherence"** hypothesis proposes that neurons communicate effectively when their oscillations are phase-aligned — allowing information to arrive at optimal moments in the receiving neuron's excitability cycle.

Many of the key connectivity metrics we'll study are built on phase relationships:
- **PLV** (Phase Locking Value): measures consistency of phase difference
- **PLI** (Phase Lag Index): considers the direction of phase lead/lag
- **wPLI** (weighted PLI): weights by the magnitude of phase difference

However, phase is fundamentally different from ordinary numbers — **it is circular**. An angle of 359° is very close to 1°, even though numerically they seem far apart. Standard statistics like mean and variance do not work correctly on circular data.

In this notebook, we build the mathematical tools for proper phase analysis:
- Phase wrapping and unwrapping
- Phase differences between signals
- Circular statistics (mean, variance, concentration)
- Visualization techniques for phase data

These tools form the foundation for all phase-based connectivity metrics.

---

## 2. Phase as a Circular Variable

Phase is an **angle**, typically represented in radians within the range [-π, π] or [0, 2π]. The key property that makes phase special is that it **wraps around**: a phase of π + 0.1 is actually very close to -π + 0.1, because they represent nearly the same position on the circle.

**Analogy: Clock times**

Consider the times 11:55 PM and 00:05 AM. Numerically, they seem very different (almost 12 hours apart), but in reality they are only 10 minutes apart. This is exactly how phase works!

**The averaging problem**

Linear arithmetic fails with circular data. Consider two phase values:
- φ₁ = -0.9π (just below -π, on the "negative" side)
- φ₂ = +0.9π (just above π, on the "positive" side)

The linear mean is (−0.9π + 0.9π) / 2 = 0. But this is completely wrong! Both phases are near π (or equivalently -π), so the mean should be approximately ±π.

We need **circular (directional) statistics** to handle phase correctly.

In [ ]:
# ============================================================================
# VISUALIZATION 1: Phase as a Point on the Unit Circle
# ============================================================================

# Create figure with unit circle
fig, ax = plt.subplots(figsize=(8, 8))

# Draw unit circle
theta_circle = np.linspace(0, 2 * np.pi, 100)
ax.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', linewidth=1.5, alpha=0.3)

# Draw reference lines
ax.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
ax.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)

# Define some phase values to show
phase_values = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, -3*np.pi/4, -np.pi/2, -np.pi/4]
phase_labels = ['0', 'π/4', 'π/2', '3π/4', 'π', '-3π/4', '-π/2', '-π/4']
colors = [PRIMARY_BLUE, ACCENT_TEAL, PRIMARY_GREEN, ACCENT_GOLD, 
          PRIMARY_RED, ACCENT_PURPLE, SECONDARY_PINK, PRIMARY_BLUE]

# Plot each phase as a point on the circle
for phase, label, color in zip(phase_values, phase_labels, colors):
    x, y = np.cos(phase), np.sin(phase)
    ax.scatter(x, y, s=150, c=color, zorder=5, edgecolors='white', linewidths=2)
    # Add label with offset
    offset = 1.15
    ax.annotate(f'φ = {label}', xy=(x, y), xytext=(offset * x, offset * y),
                fontsize=10, ha='center', va='center')

# Highlight that π and -π are the same point!
ax.annotate('π and -π are the\nSAME point!', 
            xy=(-1, 0), xytext=(-0.6, -0.2),
            fontsize=11, color=PRIMARY_RED, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))

# Draw a phase near the boundary to show closeness
phase_near_pi_pos = 0.95 * np.pi
phase_near_pi_neg = -0.95 * np.pi
x1, y1 = np.cos(phase_near_pi_pos), np.sin(phase_near_pi_pos)
x2, y2 = np.cos(phase_near_pi_neg), np.sin(phase_near_pi_neg)

ax.scatter([x1, x2], [y1, y2], s=100, c='orange', marker='D', zorder=6, edgecolors='black')
ax.annotate('+0.95π', xy=(x1, y1), xytext=(x1 - 0.3, y1 + 0.2), fontsize=9)
ax.annotate('-0.95π', xy=(x2, y2), xytext=(x2 - 0.3, y2 - 0.2), fontsize=9)

# Draw arc showing they are close
arc_theta = np.linspace(phase_near_pi_pos, 2*np.pi + phase_near_pi_neg, 20)
ax.plot(1.05 * np.cos(arc_theta), 1.05 * np.sin(arc_theta), 
        color='orange', linewidth=2, linestyle='--')
ax.annotate('Only 0.1π apart!', xy=(-1.1, 0), xytext=(-1.3, 0.25),
            fontsize=10, color='orange', fontweight='bold')

ax.set_xlim(-1.8, 1.8)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_title('Visualization 1: Phase as a Point on the Unit Circle', 
             fontsize=13, fontweight='bold', pad=15)
ax.set_xlabel('cos(φ)', fontsize=11)
ax.set_ylabel('sin(φ)', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Phase 'wraps around' — values near π and -π are very close!")

In [ ]:
# ============================================================================
# VISUALIZATION 2: The Circular Mean Problem
# ============================================================================

# Two phases that are both near π
phi1 = -0.9 * np.pi  # Just below π on negative side
phi2 = 0.9 * np.pi   # Just above π on positive side

# Linear mean (WRONG)
linear_mean = (phi1 + phi2) / 2  # = 0

# Circular mean (CORRECT)
# Convert to unit vectors, average, convert back
mean_x = (np.cos(phi1) + np.cos(phi2)) / 2
mean_y = (np.sin(phi1) + np.sin(phi2)) / 2
circular_mean = np.arctan2(mean_y, mean_x)  # ≈ π or -π

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ---- Left panel: Linear mean (wrong) ----
ax1 = axes[0]

# Draw unit circle
theta_circle = np.linspace(0, 2 * np.pi, 100)
ax1.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', linewidth=1.5, alpha=0.3)
ax1.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
ax1.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)

# Plot the two phases
x1, y1 = np.cos(phi1), np.sin(phi1)
x2, y2 = np.cos(phi2), np.sin(phi2)
ax1.scatter([x1, x2], [y1, y2], s=200, c=[PRIMARY_BLUE, PRIMARY_GREEN], 
            zorder=5, edgecolors='white', linewidths=2)
ax1.annotate(f'φ₁ = -0.9π', xy=(x1, y1), xytext=(x1 - 0.4, y1 - 0.25), fontsize=11)
ax1.annotate(f'φ₂ = +0.9π', xy=(x2, y2), xytext=(x2 - 0.4, y2 + 0.2), fontsize=11)

# Plot linear mean (WRONG) at phase = 0
x_lin = np.cos(linear_mean)
y_lin = np.sin(linear_mean)
ax1.scatter(x_lin, y_lin, s=300, c=PRIMARY_RED, marker='X', zorder=6, 
            edgecolors='black', linewidths=2)
ax1.annotate(f'Linear mean = 0\n(WRONG!)', xy=(x_lin, y_lin), 
             xytext=(0.5, 0.4), fontsize=11, color=PRIMARY_RED, fontweight='bold',
             arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))

ax1.set_xlim(-1.6, 1.6)
ax1.set_ylim(-1.4, 1.4)
ax1.set_aspect('equal')
ax1.set_title('Linear Mean: (φ₁ + φ₂) / 2 = 0', fontsize=12, fontweight='bold', color=PRIMARY_RED)
ax1.grid(True, alpha=0.3)

# ---- Right panel: Circular mean (correct) ----
ax2 = axes[1]

# Draw unit circle
ax2.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', linewidth=1.5, alpha=0.3)
ax2.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
ax2.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)

# Plot the two phases as vectors
ax2.arrow(0, 0, 0.9*x1, 0.9*y1, head_width=0.08, head_length=0.05, 
          fc=PRIMARY_BLUE, ec=PRIMARY_BLUE, linewidth=2)
ax2.arrow(0, 0, 0.9*x2, 0.9*y2, head_width=0.08, head_length=0.05, 
          fc=PRIMARY_GREEN, ec=PRIMARY_GREEN, linewidth=2)

# Plot mean vector
mean_length = np.sqrt(mean_x**2 + mean_y**2)
ax2.arrow(0, 0, 0.9*mean_x/mean_length if mean_length > 0 else 0, 
          0.9*mean_y/mean_length if mean_length > 0 else 0, 
          head_width=0.08, head_length=0.05, 
          fc=ACCENT_GOLD, ec=ACCENT_GOLD, linewidth=3)

# Plot resultant (sum of vectors, not normalized)
ax2.arrow(0, 0, mean_x, mean_y, head_width=0.05, head_length=0.03,
          fc='gray', ec='gray', linewidth=1, alpha=0.5, linestyle='--')

# Plot circular mean point
x_circ = np.cos(circular_mean)
y_circ = np.sin(circular_mean)
ax2.scatter(x_circ, y_circ, s=300, c=ACCENT_GOLD, marker='*', zorder=6, 
            edgecolors='black', linewidths=1)

ax2.annotate(f'Circular mean ≈ π\n(CORRECT!)', xy=(x_circ, y_circ), 
             xytext=(-1.4, 0.5), fontsize=11, color=PRIMARY_GREEN, fontweight='bold',
             arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN))

ax2.set_xlim(-1.6, 1.6)
ax2.set_ylim(-1.4, 1.4)
ax2.set_aspect('equal')
ax2.set_title('Circular Mean: Average Unit Vectors → arctan2', 
              fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
ax2.grid(True, alpha=0.3)

plt.suptitle('Visualization 2: Why Linear Averaging Fails for Phase', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"φ₁ = {phi1/np.pi:.2f}π, φ₂ = {phi2/np.pi:.2f}π")
print(f"Linear mean: {linear_mean/np.pi:.2f}π (completely wrong!)")
print(f"Circular mean: {circular_mean/np.pi:.2f}π (correct - both phases are near π)")

---

## 3. Phase Wrapping and Unwrapping

When we extract phase from a signal, it is typically **wrapped** to the range [-π, π]. This means that as the phase naturally increases through a cycle, it suddenly "jumps" from π back to -π. This is purely a representation artifact — the phase is actually continuous.

**Phase wrapping**: The discontinuous jumps from π to -π (or vice versa)

**Phase unwrapping**: Removing these artificial jumps to create a continuous phase signal

Unwrapping is essential for:
- Computing **instantaneous frequency** (derivative of phase)
- Visualizing **phase evolution** over time
- Tracking **cumulative phase** across many cycles

The `np.unwrap()` function adds or subtracts 2π at each jump to create continuity.

**Caution**: Unwrapping can fail with very noisy or undersampled signals, as it may misinterpret noise as phase jumps.

In [ ]:
# ============================================================================
# FUNCTIONS: Phase Wrapping and Unwrapping
# ============================================================================

def wrap_phase(phase: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Wrap phase values to the range [-π, π].
    
    Parameters
    ----------
    phase : NDArray[np.float64]
        Phase values in radians (can be any range).
        
    Returns
    -------
    NDArray[np.float64]
        Phase values wrapped to [-π, π].
        
    Examples
    --------
    >>> wrap_phase(np.array([3.5, -3.5, 7.0]))
    array([-2.78..., 2.78..., 0.71...])
    """
    return np.angle(np.exp(1j * phase))


def unwrap_phase(phase: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Unwrap phase to create a continuous signal.
    
    Removes discontinuities by adding ±2π at each jump.
    
    Parameters
    ----------
    phase : NDArray[np.float64]
        Wrapped phase values in radians.
        
    Returns
    -------
    NDArray[np.float64]
        Unwrapped (continuous) phase values.
        
    Notes
    -----
    Unwrapping may fail with very noisy or undersampled signals.
    """
    return np.unwrap(phase)


print("✓ wrap_phase() and unwrap_phase() functions defined")

In [ ]:
# ============================================================================
# VISUALIZATION 3: Wrapped vs Unwrapped Phase
# ============================================================================

# Create a long sine wave (multiple cycles)
fs = 250
duration = 2.0  # 2 seconds
t = np.arange(0, duration, 1/fs)
freq = 5  # 5 Hz = 10 cycles in 2 seconds

# Generate signal and extract phase
signal = np.sin(2 * np.pi * freq * t)
analytic = hilbert(signal)
phase_wrapped = np.angle(analytic)
phase_unwrapped = np.unwrap(phase_wrapped)

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(12, 9))

# Signal
axes[0].plot(t * 1000, signal, color=PRIMARY_BLUE, linewidth=1.5)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('5 Hz Sine Wave (10 cycles in 2 seconds)', fontsize=12, fontweight='bold')
axes[0].set_xlim(0, 2000)
axes[0].grid(True, alpha=0.3)

# Wrapped phase (sawtooth pattern)
axes[1].plot(t * 1000, phase_wrapped, color=ACCENT_PURPLE, linewidth=1.5)
axes[1].axhline(y=np.pi, color='gray', linestyle='--', alpha=0.5)
axes[1].axhline(y=-np.pi, color='gray', linestyle='--', alpha=0.5)
axes[1].set_ylabel('Phase (rad)', fontsize=11)
axes[1].set_title('Wrapped Phase: Jumps from π to -π (sawtooth pattern)', 
                  fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[1].set_xlim(0, 2000)
axes[1].set_ylim(-4, 4)
axes[1].grid(True, alpha=0.3)

# Annotate a jump
jump_idx = np.where(np.abs(np.diff(phase_wrapped)) > 5)[0][0]
axes[1].annotate('Phase jump!', xy=(t[jump_idx] * 1000, phase_wrapped[jump_idx]),
                 xytext=(t[jump_idx] * 1000 + 100, 2.5), fontsize=10, color=PRIMARY_RED,
                 arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))

# Unwrapped phase (continuous)
axes[2].plot(t * 1000, phase_unwrapped, color=PRIMARY_GREEN, linewidth=1.5)
axes[2].set_xlabel('Time (ms)', fontsize=11)
axes[2].set_ylabel('Phase (rad)', fontsize=11)
axes[2].set_title('Unwrapped Phase: Continuous, monotonically increasing', 
                  fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[2].set_xlim(0, 2000)
axes[2].grid(True, alpha=0.3)

# Add reference lines for multiples of 2π
for k in range(1, 11):
    axes[2].axhline(y=k * 2 * np.pi, color='gray', linestyle=':', alpha=0.3)

axes[2].annotate(f'Total: {phase_unwrapped[-1]/(2*np.pi):.1f} cycles', 
                 xy=(1800, phase_unwrapped[-1]), fontsize=10,
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('Visualization 3: Phase Wrapping vs Unwrapping', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Signal: {freq} Hz for {duration} s = {freq * duration:.0f} complete cycles")
print(f"Final unwrapped phase: {phase_unwrapped[-1]:.2f} rad = {phase_unwrapped[-1]/(2*np.pi):.1f} × 2π")

In [ ]:
# ============================================================================
# VISUALIZATION 4: When Unwrapping Fails (Noisy Signal)
# ============================================================================

# Create a noisy narrowband signal
np.random.seed(42)
fs_noisy = 250
duration_noisy = 1.0
t_noisy = np.arange(0, duration_noisy, 1/fs_noisy)
freq_noisy = 10

# Clean signal
clean_signal = np.sin(2 * np.pi * freq_noisy * t_noisy)

# Add substantial noise
noise_level = 1.5  # High noise level
noisy_signal = clean_signal + noise_level * np.random.randn(len(t_noisy))

# Filter to narrow band (should help but noise still affects phase)
noisy_filtered = bandpass_filter(noisy_signal, 8, 12, fs_noisy)

# Extract phases
phase_clean = np.angle(hilbert(clean_signal))
phase_noisy = np.angle(hilbert(noisy_filtered))

# Unwrap both
phase_clean_unwrapped = np.unwrap(phase_clean)
phase_noisy_unwrapped = np.unwrap(phase_noisy)

# Expected unwrapped phase (theoretical)
expected_unwrapped = 2 * np.pi * freq_noisy * t_noisy

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Clean signal - wrapped
axes[0, 0].plot(t_noisy * 1000, phase_clean, color=PRIMARY_GREEN, linewidth=1)
axes[0, 0].set_ylabel('Phase (rad)', fontsize=10)
axes[0, 0].set_title('Clean Signal: Wrapped Phase', fontsize=11, fontweight='bold')
axes[0, 0].set_ylim(-4, 4)
axes[0, 0].grid(True, alpha=0.3)

# Clean signal - unwrapped
axes[0, 1].plot(t_noisy * 1000, phase_clean_unwrapped, color=PRIMARY_GREEN, linewidth=1.5, label='Unwrapped')
axes[0, 1].plot(t_noisy * 1000, expected_unwrapped, 'k--', alpha=0.5, label='Expected')
axes[0, 1].set_ylabel('Phase (rad)', fontsize=10)
axes[0, 1].set_title('Clean Signal: Unwrapped Phase ✓', fontsize=11, fontweight='bold', color=PRIMARY_GREEN)
axes[0, 1].legend(loc='upper left')
axes[0, 1].grid(True, alpha=0.3)

# Noisy signal - wrapped
axes[1, 0].plot(t_noisy * 1000, phase_noisy, color=PRIMARY_RED, linewidth=1)
axes[1, 0].set_xlabel('Time (ms)', fontsize=10)
axes[1, 0].set_ylabel('Phase (rad)', fontsize=10)
axes[1, 0].set_title('Noisy Signal: Wrapped Phase (erratic)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylim(-4, 4)
axes[1, 0].grid(True, alpha=0.3)

# Noisy signal - unwrapped (with errors)
axes[1, 1].plot(t_noisy * 1000, phase_noisy_unwrapped, color=PRIMARY_RED, linewidth=1.5, label='Unwrapped (with errors)')
axes[1, 1].plot(t_noisy * 1000, expected_unwrapped, 'k--', alpha=0.5, label='Expected')
axes[1, 1].set_xlabel('Time (ms)', fontsize=10)
axes[1, 1].set_ylabel('Phase (rad)', fontsize=10)
axes[1, 1].set_title('Noisy Signal: Unwrapping Accumulates Errors ✗', 
                     fontsize=11, fontweight='bold', color=PRIMARY_RED)
axes[1, 1].legend(loc='upper left')
axes[1, 1].grid(True, alpha=0.3)

# Calculate error
final_error = np.abs(phase_noisy_unwrapped[-1] - expected_unwrapped[-1])
axes[1, 1].annotate(f'Error: {final_error:.1f} rad\n({final_error/(2*np.pi):.1f} cycles)', 
                    xy=(900, phase_noisy_unwrapped[-1]), fontsize=10,
                    bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_RED))

plt.suptitle('Visualization 4: Unwrapping Failure with Noisy Signals', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("⚠️ Lesson: Unwrapping is reliable for clean signals, but accumulates errors with noise.")
print("   For noisy EEG, prefer working with wrapped phase or use phase differences.")

---

## 4. Phase Difference Between Two Signals

The **phase difference** between two signals is the key quantity for understanding synchronization:

$$\Delta\phi(t) = \phi_1(t) - \phi_2(t)$$

The result should always be **wrapped** to [-π, π] to avoid ambiguity.

**Interpretation of phase difference**:
- **Δφ ≈ 0**: Signals are **in-phase** (synchronized, peaks align)
- **Δφ ≈ ±π**: Signals are **anti-phase** (one peaks when other troughs)
- **Δφ ≈ ±π/2**: Signals are in **quadrature** (90° offset)

**Key insight for connectivity**:
- **Constant phase difference** over time → signals are **phase-locked** (synchronized)
- **Varying phase difference** → signals are **not synchronized**

This is the foundation of PLV: it measures how *consistent* the phase difference is over time.

In [ ]:
# ============================================================================
# FUNCTION: Compute Phase Difference
# ============================================================================

def compute_phase_difference(phase1: NDArray[np.float64], 
                              phase2: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Compute the phase difference between two signals, wrapped to [-π, π].
    
    Parameters
    ----------
    phase1 : NDArray[np.float64]
        Phase of first signal in radians.
    phase2 : NDArray[np.float64]
        Phase of second signal in radians.
        
    Returns
    -------
    NDArray[np.float64]
        Phase difference (phase1 - phase2) wrapped to [-π, π].
        
    Examples
    --------
    >>> phase1 = np.array([0.1, 0.2, 0.3])
    >>> phase2 = np.array([0.0, 0.1, 0.2])
    >>> compute_phase_difference(phase1, phase2)
    array([0.1, 0.1, 0.1])  # Constant difference = synchronized
    """
    diff = phase1 - phase2
    return wrap_phase(diff)


print("✓ compute_phase_difference() function defined")

In [ ]:
# ============================================================================
# VISUALIZATION 5: Synchronized Signals (Constant Phase Difference)
# ============================================================================

# Create two synchronized signals with constant phase offset
fs_sync = 250
duration_sync = 1.0
t_sync = np.arange(0, duration_sync, 1/fs_sync)
freq_sync = 10  # Hz

# Constant phase offset of π/3 (60°)
phase_offset_constant = np.pi / 3

signal1_sync = np.sin(2 * np.pi * freq_sync * t_sync)
signal2_sync = np.sin(2 * np.pi * freq_sync * t_sync + phase_offset_constant)

# Extract phases
phase1_sync = np.angle(hilbert(signal1_sync))
phase2_sync = np.angle(hilbert(signal2_sync))

# Compute phase difference
phase_diff_sync = compute_phase_difference(phase1_sync, phase2_sync)

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(12, 9))

# Both signals overlaid
axes[0].plot(t_sync * 1000, signal1_sync, color=PRIMARY_BLUE, linewidth=1.5, label='Signal 1')
axes[0].plot(t_sync * 1000, signal2_sync, color=SECONDARY_PINK, linewidth=1.5, label='Signal 2 (+60°)')
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Two 10 Hz Signals with Constant Phase Offset (60°)', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].set_xlim(0, 500)
axes[0].grid(True, alpha=0.3)

# Both phases
axes[1].plot(t_sync * 1000, phase1_sync, color=PRIMARY_BLUE, linewidth=1.5, label='Phase 1')
axes[1].plot(t_sync * 1000, phase2_sync, color=SECONDARY_PINK, linewidth=1.5, label='Phase 2')
axes[1].set_ylabel('Phase (rad)', fontsize=11)
axes[1].set_title('Instantaneous Phases (both show sawtooth pattern)', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].set_xlim(0, 500)
axes[1].set_ylim(-4, 4)
axes[1].grid(True, alpha=0.3)

# Phase difference (should be constant!)
axes[2].plot(t_sync * 1000, phase_diff_sync, color=PRIMARY_GREEN, linewidth=2)
axes[2].axhline(y=-phase_offset_constant, color='gray', linestyle='--', 
                label=f'Expected: -π/3 = {-phase_offset_constant:.2f} rad')
axes[2].set_xlabel('Time (ms)', fontsize=11)
axes[2].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[2].set_title('Phase Difference: CONSTANT → Signals are SYNCHRONIZED', 
                  fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[2].legend(loc='upper right')
axes[2].set_xlim(0, 500)
axes[2].set_ylim(-2, 2)
axes[2].grid(True, alpha=0.3)

plt.suptitle('Visualization 5: Phase-Locked (Synchronized) Signals', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Phase difference is constant at {np.mean(phase_diff_sync):.3f} rad ({np.rad2deg(np.mean(phase_diff_sync)):.1f}°)")
print("→ These signals are perfectly phase-locked!")

In [ ]:
# ============================================================================
# VISUALIZATION 6: Non-Synchronized Signals (Drifting Phase Difference)
# ============================================================================

# Create two signals with DIFFERENT frequencies (→ drifting phase)
fs_drift = 250
duration_drift = 1.0
t_drift = np.arange(0, duration_drift, 1/fs_drift)

freq1_drift = 10.0  # Hz
freq2_drift = 10.5  # Slightly different → phase drift

signal1_drift = np.sin(2 * np.pi * freq1_drift * t_drift)
signal2_drift = np.sin(2 * np.pi * freq2_drift * t_drift)

# Extract phases
phase1_drift = np.angle(hilbert(signal1_drift))
phase2_drift = np.angle(hilbert(signal2_drift))

# Compute phase difference
phase_diff_drift = compute_phase_difference(phase1_drift, phase2_drift)

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(12, 9))

# Both signals overlaid
axes[0].plot(t_drift * 1000, signal1_drift, color=PRIMARY_BLUE, linewidth=1.5, label='Signal 1 (10.0 Hz)')
axes[0].plot(t_drift * 1000, signal2_drift, color=SECONDARY_PINK, linewidth=1.5, label='Signal 2 (10.5 Hz)')
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Two Signals with Slightly Different Frequencies', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].set_xlim(0, 1000)
axes[0].grid(True, alpha=0.3)

# Both phases
axes[1].plot(t_drift * 1000, phase1_drift, color=PRIMARY_BLUE, linewidth=1.5, label='Phase 1')
axes[1].plot(t_drift * 1000, phase2_drift, color=SECONDARY_PINK, linewidth=1.5, label='Phase 2')
axes[1].set_ylabel('Phase (rad)', fontsize=11)
axes[1].set_title('Instantaneous Phases', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].set_xlim(0, 1000)
axes[1].set_ylim(-4, 4)
axes[1].grid(True, alpha=0.3)

# Phase difference (drifting!)
axes[2].plot(t_drift * 1000, phase_diff_drift, color=PRIMARY_RED, linewidth=2)
axes[2].set_xlabel('Time (ms)', fontsize=11)
axes[2].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[2].set_title('Phase Difference: DRIFTING → Signals are NOT SYNCHRONIZED', 
                  fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[2].set_xlim(0, 1000)
axes[2].set_ylim(-4, 4)
axes[2].grid(True, alpha=0.3)

# Annotate the drift
axes[2].annotate('Phase drifts through\nfull cycle (2π)', xy=(500, 0), xytext=(700, 2),
                 fontsize=10, color=PRIMARY_RED,
                 arrowprops=dict(arrowstyle='->', color=PRIMARY_RED))

plt.suptitle('Visualization 6: Non-Synchronized Signals (Different Frequencies)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Frequency difference: {freq2_drift - freq1_drift} Hz")
print(f"Phase difference varies from {np.min(phase_diff_drift):.2f} to {np.max(phase_diff_drift):.2f} rad")
print("→ These signals are NOT synchronized (phase drifts through full cycle)")

---

## 5. Circular Mean

As we saw earlier, computing the average of phase values requires special treatment. The **circular mean** algorithm:

1. Convert each phase to a **unit vector**: $(cos(\phi), sin(\phi))$
2. **Average** the x and y components separately
3. Convert back to angle: $\bar{\phi} = atan2(\bar{y}, \bar{x})$

The result is the **preferred direction** of the phase distribution — the angle where phases tend to cluster.

**Important**: The *magnitude* of the resultant vector (before normalizing) tells us how concentrated the distribution is. This will be key for understanding PLV!

In [ ]:
# ============================================================================
# FUNCTION: Circular Mean
# ============================================================================

def circular_mean(phases: NDArray[np.float64]) -> float:
    """
    Compute the circular (directional) mean of phase values.
    
    Uses the vector averaging method:
    1. Convert phases to unit vectors
    2. Average x and y components
    3. Return angle of resultant vector
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
        
    Returns
    -------
    float
        Circular mean in radians, range [-π, π].
        
    Notes
    -----
    The circular mean is undefined when R = 0 (uniform distribution).
    In this case, the function returns 0, but this should be interpreted
    with caution.
    
    Examples
    --------
    >>> circular_mean(np.array([0.1, 0.2, 0.15]))
    0.15  # Phases clustered around 0.15
    >>> circular_mean(np.array([-0.9*np.pi, 0.9*np.pi]))
    ≈ π  # Both near π, not 0!
    """
    mean_cos = np.mean(np.cos(phases))
    mean_sin = np.mean(np.sin(phases))
    return np.arctan2(mean_sin, mean_cos)


print("✓ circular_mean() function defined")

In [ ]:
# ============================================================================
# VISUALIZATION 7: Circular Mean Illustrated
# ============================================================================

# Generate phases clustered around π/4 with some spread
np.random.seed(42)
n_phases = 30
center_phase = np.pi / 4
spread = 0.3  # radians

# Use von Mises distribution (circular equivalent of Gaussian)
from scipy.stats import vonmises
phases_clustered = vonmises.rvs(kappa=1/spread**2, loc=center_phase, size=n_phases)

# Wrap to [-π, π]
phases_clustered = wrap_phase(phases_clustered)

# Compute circular mean
circ_mean = circular_mean(phases_clustered)

# Compute resultant vector
mean_cos = np.mean(np.cos(phases_clustered))
mean_sin = np.mean(np.sin(phases_clustered))
R = np.sqrt(mean_cos**2 + mean_sin**2)

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Unit circle with phases
ax1 = axes[0]
theta_circle = np.linspace(0, 2 * np.pi, 100)
ax1.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', linewidth=1.5, alpha=0.3)
ax1.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
ax1.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)

# Plot individual phases as points
for phi in phases_clustered:
    ax1.scatter(np.cos(phi), np.sin(phi), s=80, c=PRIMARY_BLUE, alpha=0.6, 
                edgecolors='white', linewidths=0.5)

# Plot individual unit vectors (faint)
for phi in phases_clustered:
    ax1.arrow(0, 0, 0.85*np.cos(phi), 0.85*np.sin(phi), 
              head_width=0.03, head_length=0.02, fc=PRIMARY_BLUE, ec=PRIMARY_BLUE, alpha=0.2)

# Plot resultant vector (sum)
ax1.arrow(0, 0, mean_cos, mean_sin, head_width=0.06, head_length=0.04,
          fc=ACCENT_GOLD, ec=ACCENT_GOLD, linewidth=2, label=f'Resultant (R={R:.2f})')

# Plot circular mean direction
ax1.arrow(0, 0, 0.95*np.cos(circ_mean), 0.95*np.sin(circ_mean), 
          head_width=0.06, head_length=0.04, fc=PRIMARY_GREEN, ec=PRIMARY_GREEN, linewidth=3)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.3, 1.3)
ax1.set_aspect('equal')
ax1.set_title(f'Circular Mean = {np.rad2deg(circ_mean):.1f}° (R = {R:.2f})', 
              fontsize=12, fontweight='bold')
ax1.set_xlabel('cos(φ)', fontsize=11)
ax1.set_ylabel('sin(φ)', fontsize=11)
ax1.grid(True, alpha=0.3)

# Legend
ax1.scatter([], [], s=80, c=PRIMARY_BLUE, label='Individual phases')
ax1.plot([], [], color=ACCENT_GOLD, linewidth=2, label=f'Resultant vector (R={R:.2f})')
ax1.plot([], [], color=PRIMARY_GREEN, linewidth=3, label=f'Circular mean direction')
ax1.legend(loc='lower left', fontsize=9)

# Right: Dispersed distribution for comparison
ax2 = axes[1]

# Generate uniform-ish distribution
phases_dispersed = np.random.uniform(-np.pi, np.pi, n_phases)
circ_mean_disp = circular_mean(phases_dispersed)
mean_cos_disp = np.mean(np.cos(phases_dispersed))
mean_sin_disp = np.mean(np.sin(phases_dispersed))
R_disp = np.sqrt(mean_cos_disp**2 + mean_sin_disp**2)

ax2.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', linewidth=1.5, alpha=0.3)
ax2.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
ax2.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)

for phi in phases_dispersed:
    ax2.scatter(np.cos(phi), np.sin(phi), s=80, c=PRIMARY_RED, alpha=0.6,
                edgecolors='white', linewidths=0.5)
    ax2.arrow(0, 0, 0.85*np.cos(phi), 0.85*np.sin(phi),
              head_width=0.03, head_length=0.02, fc=PRIMARY_RED, ec=PRIMARY_RED, alpha=0.2)

# Resultant (very short for uniform)
ax2.arrow(0, 0, mean_cos_disp * 3, mean_sin_disp * 3, head_width=0.06, head_length=0.04,
          fc=ACCENT_GOLD, ec=ACCENT_GOLD, linewidth=2)

ax2.set_xlim(-1.5, 1.5)
ax2.set_ylim(-1.3, 1.3)
ax2.set_aspect('equal')
ax2.set_title(f'Dispersed: R = {R_disp:.2f} (mean less meaningful)', 
              fontsize=12, fontweight='bold', color=PRIMARY_RED)
ax2.set_xlabel('cos(φ)', fontsize=11)
ax2.set_ylabel('sin(φ)', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.suptitle('Visualization 7: Circular Mean and Resultant Vector Length (R)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Concentrated distribution: R = {R:.2f} (high concentration, mean is meaningful)")
print(f"Dispersed distribution: R = {R_disp:.2f} (low concentration, mean is less meaningful)")

---

## 6. Circular Variance and Concentration

The **resultant vector length (R)** is the key measure of phase concentration:

$$R = \left| \frac{1}{N} \sum_{n=1}^{N} e^{i\phi_n} \right| = \sqrt{\bar{cos}^2 + \bar{sin}^2}$$

**Interpretation**:
- **R = 1**: All phases are identical (perfect concentration)
- **R = 0**: Phases are uniformly distributed (no preferred direction)
- **0 < R < 1**: Partial concentration

Related measures:
- **Circular variance**: $V = 1 - R$ (higher = more spread)
- **Circular standard deviation**: $\sigma = \sqrt{-2 \ln R}$

**Key insight**: R is exactly what **PLV measures**! PLV = R of the phase difference distribution between two signals.

In [ ]:
# ============================================================================
# FUNCTIONS: Circular Variance and Related Measures
# ============================================================================

def resultant_vector_length(phases: NDArray[np.float64]) -> float:
    """
    Compute the resultant vector length (R) of a phase distribution.
    
    R measures the concentration of phases:
    - R = 1: all phases identical
    - R = 0: uniform distribution
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
        
    Returns
    -------
    float
        Resultant vector length R, range [0, 1].
        
    Notes
    -----
    This is equivalent to PLV when applied to phase differences!
    """
    return np.abs(np.mean(np.exp(1j * phases)))


def circular_variance(phases: NDArray[np.float64]) -> float:
    """
    Compute the circular variance of a phase distribution.
    
    Circular variance V = 1 - R, where R is the resultant vector length.
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
        
    Returns
    -------
    float
        Circular variance V, range [0, 1].
        Higher values indicate more dispersion.
    """
    return 1.0 - resultant_vector_length(phases)


def circular_std(phases: NDArray[np.float64]) -> float:
    """
    Compute the circular standard deviation of a phase distribution.
    
    Circular std = sqrt(-2 * ln(R)), where R is the resultant vector length.
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
        
    Returns
    -------
    float
        Circular standard deviation in radians.
        
    Notes
    -----
    Returns inf if R = 0 (uniform distribution).
    """
    R = resultant_vector_length(phases)
    if R == 0:
        return np.inf
    return np.sqrt(-2 * np.log(R))


print("✓ resultant_vector_length(), circular_variance(), circular_std() defined")

In [ ]:
# ============================================================================
# VISUALIZATION 8: Three Levels of Phase Concentration
# ============================================================================

from scipy.stats import vonmises

# Create three distributions with different concentrations
np.random.seed(123)
n_samples = 50

# High concentration (κ = 10)
kappa_high = 10
phases_high = vonmises.rvs(kappa=kappa_high, loc=np.pi/3, size=n_samples)

# Medium concentration (κ = 2)
kappa_med = 2
phases_med = vonmises.rvs(kappa=kappa_med, loc=np.pi/3, size=n_samples)

# Low concentration (nearly uniform, κ = 0.1)
kappa_low = 0.1
phases_low = vonmises.rvs(kappa=kappa_low, loc=np.pi/3, size=n_samples)

# Compute R for each
R_high = resultant_vector_length(phases_high)
R_med = resultant_vector_length(phases_med)
R_low = resultant_vector_length(phases_low)

# Create figure with 3 polar plots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw=dict(projection='polar'))

distributions = [
    (phases_high, R_high, 'High Concentration', PRIMARY_GREEN, kappa_high),
    (phases_med, R_med, 'Medium Concentration', ACCENT_GOLD, kappa_med),
    (phases_low, R_low, 'Low Concentration', PRIMARY_RED, kappa_low)
]

for ax, (phases, R, title, color, kappa) in zip(axes, distributions):
    # Plot phases as points on unit circle
    ax.scatter(phases, np.ones_like(phases), s=50, c=color, alpha=0.7, edgecolors='white')
    
    # Plot mean direction
    mean_dir = circular_mean(phases)
    ax.annotate('', xy=(mean_dir, R), xytext=(mean_dir, 0),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    
    # Draw R as radial distance
    ax.plot([mean_dir, mean_dir], [0, R], 'k-', linewidth=2)
    ax.scatter(mean_dir, R, s=100, c='black', zorder=5)
    
    ax.set_ylim(0, 1.2)
    ax.set_rticks([0.5, 1.0])
    ax.set_title(f'{title}\nR = {R:.2f}', fontsize=12, fontweight='bold', 
                 color=color, pad=15)
    
    # Add variance info
    V = circular_variance(phases)
    ax.text(0, 0.3, f'V = {V:.2f}', ha='center', fontsize=10, 
            transform=ax.transAxes)

plt.suptitle('Visualization 8: Resultant Vector Length (R) Measures Phase Concentration', 
             fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("Interpretation of R:")
print(f"  High concentration: R = {R_high:.2f} → Phases cluster tightly")
print(f"  Medium concentration: R = {R_med:.2f} → Some clustering")
print(f"  Low concentration: R = {R_low:.2f} → Nearly uniform (no preferred direction)")
print("\n💡 When applied to phase differences, R becomes PLV!")

---

## 7. Visualizing Phase Distributions

Phase data requires specialized visualization techniques because of its circular nature:

1. **Linear histogram**: Simple but misleading at boundaries (phases near π and -π appear far apart)
2. **Polar histogram (rose plot)**: Natural circular representation
3. **Scatter on unit circle**: Shows individual phase values
4. **Mean vector overlay**: Shows preferred direction and concentration

Let's create reusable plotting functions and compare these approaches.

In [ ]:
# ============================================================================
# FUNCTIONS: Phase Visualization
# ============================================================================

def plot_phase_polar_histogram(phases: NDArray[np.float64], 
                                n_bins: int = 36,
                                ax: Optional[plt.Axes] = None,
                                color: str = PRIMARY_BLUE,
                                title: str = '') -> plt.Axes:
    """
    Create a polar histogram (rose plot) of phase values.
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
    n_bins : int, optional
        Number of angular bins (default: 36 = 10° bins).
    ax : plt.Axes, optional
        Matplotlib polar axes. If None, creates new figure.
    color : str, optional
        Color for the bars.
    title : str, optional
        Title for the plot.
        
    Returns
    -------
    plt.Axes
        The matplotlib axes object.
    """
    if ax is None:
        fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
    
    # Create histogram
    bins = np.linspace(-np.pi, np.pi, n_bins + 1)
    counts, edges = np.histogram(phases, bins=bins)
    
    # Plot as bars
    width = 2 * np.pi / n_bins
    centers = (edges[:-1] + edges[1:]) / 2
    ax.bar(centers, counts, width=width, color=color, alpha=0.7, edgecolor='white')
    
    ax.set_title(title, fontsize=11, fontweight='bold', pad=10)
    return ax


def plot_phase_on_circle(phases: NDArray[np.float64],
                          ax: Optional[plt.Axes] = None,
                          show_mean: bool = True,
                          color: str = PRIMARY_BLUE,
                          title: str = '') -> plt.Axes:
    """
    Plot phases as points on the unit circle.
    
    Parameters
    ----------
    phases : NDArray[np.float64]
        Array of phase values in radians.
    ax : plt.Axes, optional
        Matplotlib axes. If None, creates new figure.
    show_mean : bool, optional
        Whether to show the mean vector (default: True).
    color : str, optional
        Color for the points.
    title : str, optional
        Title for the plot.
        
    Returns
    -------
    plt.Axes
        The matplotlib axes object.
    """
    if ax is None:
        fig, ax = plt.subplots()
    
    # Draw unit circle
    theta = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=1, alpha=0.3)
    ax.axhline(y=0, color='gray', linewidth=0.5, alpha=0.5)
    ax.axvline(x=0, color='gray', linewidth=0.5, alpha=0.5)
    
    # Plot phases as points
    x = np.cos(phases)
    y = np.sin(phases)
    ax.scatter(x, y, s=50, c=color, alpha=0.6, edgecolors='white', linewidths=0.5)
    
    if show_mean:
        mean_x = np.mean(np.cos(phases))
        mean_y = np.mean(np.sin(phases))
        R = np.sqrt(mean_x**2 + mean_y**2)
        ax.arrow(0, 0, mean_x * 0.95, mean_y * 0.95, 
                 head_width=0.05, head_length=0.03, fc='black', ec='black', linewidth=2)
        ax.text(0.05, -0.15, f'R = {R:.2f}', fontsize=10, transform=ax.transAxes)
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    return ax


print("✓ plot_phase_polar_histogram() and plot_phase_on_circle() defined")

In [ ]:
# ============================================================================
# VISUALIZATION 9: Comparing Phase Visualization Methods
# ============================================================================

# Generate phase distribution clustered near the π boundary (challenging case)
np.random.seed(456)
n_viz = 100
# Phases near π (mix of positive and negative)
phases_boundary = np.concatenate([
    vonmises.rvs(kappa=5, loc=0.95*np.pi, size=n_viz//2),
    vonmises.rvs(kappa=5, loc=-0.95*np.pi, size=n_viz//2)
])
phases_boundary = wrap_phase(phases_boundary)

# Create 2x2 comparison
fig = plt.figure(figsize=(14, 12))

# 1. Linear histogram (problematic!)
ax1 = fig.add_subplot(2, 2, 1)
ax1.hist(phases_boundary, bins=36, color=PRIMARY_BLUE, alpha=0.7, edgecolor='white')
ax1.axvline(x=np.pi, color=PRIMARY_RED, linestyle='--', linewidth=2, label='π boundary')
ax1.axvline(x=-np.pi, color=PRIMARY_RED, linestyle='--', linewidth=2)
ax1.set_xlabel('Phase (radians)', fontsize=11)
ax1.set_ylabel('Count', fontsize=11)
ax1.set_title('Linear Histogram: Misleading at Boundaries!', 
              fontsize=12, fontweight='bold', color=PRIMARY_RED)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.annotate('Appears bimodal,\nbut actually unimodal!', xy=(0, 5), fontsize=10,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 2. Polar histogram (better!)
ax2 = fig.add_subplot(2, 2, 2, projection='polar')
plot_phase_polar_histogram(phases_boundary, ax=ax2, color=PRIMARY_GREEN,
                           title='Polar Histogram: Shows True Distribution')

# 3. Unit circle scatter
ax3 = fig.add_subplot(2, 2, 3)
plot_phase_on_circle(phases_boundary, ax=ax3, color=ACCENT_PURPLE,
                     title='Unit Circle Scatter with Mean Vector')

# 4. Combined: polar with mean vector
ax4 = fig.add_subplot(2, 2, 4, projection='polar')
plot_phase_polar_histogram(phases_boundary, ax=ax4, color=ACCENT_TEAL,
                           title='Polar Histogram with Statistics')
# Add mean direction
mean_phase = circular_mean(phases_boundary)
R_viz = resultant_vector_length(phases_boundary)
ax4.annotate('', xy=(mean_phase, ax4.get_ylim()[1] * 0.8), xytext=(mean_phase, 0),
             arrowprops=dict(arrowstyle='->', color='black', lw=2))
ax4.text(0.5, -0.1, f'Mean = {np.rad2deg(mean_phase):.0f}°, R = {R_viz:.2f}', 
         fontsize=10, ha='center', transform=ax4.transAxes)

plt.suptitle('Visualization 9: Comparing Phase Visualization Methods', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Key insight: Phases near ±π appear split in linear histogram but are actually")
print("clustered together! Polar representations show the true circular structure.")

---

## 8. Phase Difference Distribution

For two signals, we compute the phase difference at each time point. The **distribution** of these phase differences reveals the synchronization between them:

- **Concentrated distribution** → signals are **phase-locked** (synchronized)
- **Uniform distribution** → signals are **independent** (not synchronized)

This is the foundation of **PLV**: it's simply the resultant vector length (R) of the phase difference distribution!

$$PLV = \left| \frac{1}{N} \sum_{n=1}^{N} e^{i \Delta\phi_n} \right| = R(\Delta\phi)$$

In [ ]:
# ============================================================================
# VISUALIZATION 10: Synchronized Signals → Concentrated Phase Difference
# ============================================================================

# Create two synchronized signals (constant phase offset)
fs_plv = 250
duration_plv = 2.0
t_plv = np.arange(0, duration_plv, 1/fs_plv)
freq_plv = 10

# Constant phase difference of π/4
offset_sync = np.pi / 4
sig1_sync = np.sin(2 * np.pi * freq_plv * t_plv)
sig2_sync = np.sin(2 * np.pi * freq_plv * t_plv + offset_sync)

# Extract phases and compute difference
phase1_plv = np.angle(hilbert(sig1_sync))
phase2_plv = np.angle(hilbert(sig2_sync))
phase_diff_plv = compute_phase_difference(phase1_plv, phase2_plv)

# Compute PLV (= R of phase difference)
PLV_sync = resultant_vector_length(phase_diff_plv)

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Phase difference over time
axes[0].plot(t_plv * 1000, phase_diff_plv, color=PRIMARY_GREEN, linewidth=1.5)
axes[0].axhline(y=-offset_sync, color='gray', linestyle='--', 
                label=f'Expected: {-offset_sync:.2f} rad')
axes[0].set_xlabel('Time (ms)', fontsize=11)
axes[0].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[0].set_title('Phase Difference Over Time (Constant)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].set_ylim(-2, 2)
axes[0].grid(True, alpha=0.3)

# Right: Polar histogram of phase difference
ax_polar = fig.add_subplot(1, 2, 2, projection='polar')
plot_phase_polar_histogram(phase_diff_plv, ax=ax_polar, color=PRIMARY_GREEN,
                           title=f'Phase Difference Distribution\nPLV = {PLV_sync:.3f}')
# Add mean vector
mean_diff = circular_mean(phase_diff_plv)
ax_polar.annotate('', xy=(mean_diff, ax_polar.get_ylim()[1] * 0.7), xytext=(mean_diff, 0),
                  arrowprops=dict(arrowstyle='->', color='black', lw=3))

plt.suptitle('Visualization 10: Synchronized Signals → High PLV', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"PLV = {PLV_sync:.3f} (very high → strong synchronization)")
print("Phase difference is concentrated around a single value.")

In [ ]:
# ============================================================================
# VISUALIZATION 11: Independent Signals → Uniform Phase Difference
# ============================================================================

# Create two independent signals (different frequencies → drifting phase)
freq1_indep = 10.0
freq2_indep = 11.0  # Different frequency

sig1_indep = np.sin(2 * np.pi * freq1_indep * t_plv)
sig2_indep = np.sin(2 * np.pi * freq2_indep * t_plv)

# Extract phases and compute difference
phase1_indep = np.angle(hilbert(sig1_indep))
phase2_indep = np.angle(hilbert(sig2_indep))
phase_diff_indep = compute_phase_difference(phase1_indep, phase2_indep)

# Compute PLV
PLV_indep = resultant_vector_length(phase_diff_indep)

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Phase difference over time (drifting)
axes[0].plot(t_plv * 1000, phase_diff_indep, color=PRIMARY_RED, linewidth=1.5)
axes[0].set_xlabel('Time (ms)', fontsize=11)
axes[0].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[0].set_title('Phase Difference Over Time (Drifting)', fontsize=12, fontweight='bold')
axes[0].set_ylim(-4, 4)
axes[0].grid(True, alpha=0.3)

# Right: Polar histogram (should be nearly uniform)
ax_polar = fig.add_subplot(1, 2, 2, projection='polar')
plot_phase_polar_histogram(phase_diff_indep, ax=ax_polar, color=PRIMARY_RED,
                           title=f'Phase Difference Distribution\nPLV = {PLV_indep:.3f}')

plt.suptitle('Visualization 11: Independent Signals → Low PLV', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"PLV = {PLV_indep:.3f} (low → weak/no synchronization)")
print("Phase difference is spread across all values (nearly uniform).")

In [ ]:
# ============================================================================
# VISUALIZATION 12: Partial Synchronization
# ============================================================================

# Create signals that are synchronized for part of the time
np.random.seed(789)

# First half: synchronized (same frequency, constant offset)
half = len(t_plv) // 2
phase_offset_partial = np.pi / 3

sig1_partial = np.sin(2 * np.pi * freq_plv * t_plv)

# Second half: independent (add random phase drift)
phase_noise = np.zeros(len(t_plv))
phase_noise[half:] = np.cumsum(np.random.randn(len(t_plv) - half) * 0.1)
sig2_partial = np.sin(2 * np.pi * freq_plv * t_plv + phase_offset_partial + phase_noise)

# Extract phases and compute difference
phase1_partial = np.angle(hilbert(sig1_partial))
phase2_partial = np.angle(hilbert(sig2_partial))
phase_diff_partial = compute_phase_difference(phase1_partial, phase2_partial)

# Compute PLV for full signal and each half
PLV_partial_full = resultant_vector_length(phase_diff_partial)
PLV_partial_first = resultant_vector_length(phase_diff_partial[:half])
PLV_partial_second = resultant_vector_length(phase_diff_partial[half:])

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top left: Signals
axes[0, 0].plot(t_plv * 1000, sig1_partial, color=PRIMARY_BLUE, alpha=0.7, label='Signal 1')
axes[0, 0].plot(t_plv * 1000, sig2_partial, color=SECONDARY_PINK, alpha=0.7, label='Signal 2')
axes[0, 0].axvline(x=1000, color='gray', linestyle='--', linewidth=2, label='Sync → Async')
axes[0, 0].set_ylabel('Amplitude', fontsize=11)
axes[0, 0].set_title('Signals: Synchronized First Half, Drifting Second Half', 
                     fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='upper right')
axes[0, 0].set_xlim(0, 2000)
axes[0, 0].grid(True, alpha=0.3)

# Top right: Phase difference over time
axes[0, 1].plot(t_plv * 1000, phase_diff_partial, color=ACCENT_PURPLE, linewidth=1)
axes[0, 1].axvline(x=1000, color='gray', linestyle='--', linewidth=2)
axes[0, 1].axhline(y=-phase_offset_partial, color='gray', linestyle=':', alpha=0.5)
axes[0, 1].fill_between(t_plv[:half] * 1000, -4, 4, alpha=0.1, color=PRIMARY_GREEN)
axes[0, 1].fill_between(t_plv[half:] * 1000, -4, 4, alpha=0.1, color=PRIMARY_RED)
axes[0, 1].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[0, 1].set_title('Phase Difference: Stable → Drifting', fontsize=12, fontweight='bold')
axes[0, 1].set_xlim(0, 2000)
axes[0, 1].set_ylim(-4, 4)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].text(250, 3, f'PLV = {PLV_partial_first:.2f}', fontsize=11, color=PRIMARY_GREEN)
axes[0, 1].text(1250, 3, f'PLV = {PLV_partial_second:.2f}', fontsize=11, color=PRIMARY_RED)

# Bottom left: Polar histogram - first half
ax_polar1 = fig.add_subplot(2, 2, 3, projection='polar')
plot_phase_polar_histogram(phase_diff_partial[:half], ax=ax_polar1, color=PRIMARY_GREEN,
                           title=f'First Half (Synchronized)\nPLV = {PLV_partial_first:.2f}')

# Bottom right: Polar histogram - second half
ax_polar2 = fig.add_subplot(2, 2, 4, projection='polar')
plot_phase_polar_histogram(phase_diff_partial[half:], ax=ax_polar2, color=PRIMARY_RED,
                           title=f'Second Half (Drifting)\nPLV = {PLV_partial_second:.2f}')

plt.suptitle('Visualization 12: Partial Synchronization (Time-Varying PLV)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Overall PLV = {PLV_partial_full:.2f}")
print(f"First half PLV = {PLV_partial_first:.2f} (synchronized)")
print(f"Second half PLV = {PLV_partial_second:.2f} (drifting)")

---

## 9. Connection to Connectivity Metrics

Now we can see the deep connection between circular statistics and connectivity metrics:

| Concept | Circular Statistics | Connectivity Metric |
|---------|---------------------|---------------------|
| Phase concentration | Resultant vector length R | **PLV** |
| Phase difference sign | Sign of sin(Δφ) | **PLI** |
| Weighted by imaginary part | \|sin(Δφ)\| weighting | **wPLI** |

**PLV (Phase Locking Value)** is simply:
$$PLV = \left| \frac{1}{N} \sum_{n=1}^{N} e^{i \Delta\phi_n} \right| = R(\Delta\phi)$$

This is exactly the `resultant_vector_length()` function we defined, applied to phase differences!

In [ ]:
# ============================================================================
# FUNCTION: Simple PLV Implementation
# ============================================================================

def compute_plv_simple(phase1: NDArray[np.float64], 
                        phase2: NDArray[np.float64]) -> float:
    """
    Compute Phase Locking Value (PLV) between two phase time series.
    
    PLV measures the consistency of phase difference over time.
    It is equivalent to the resultant vector length of phase differences.
    
    Parameters
    ----------
    phase1 : NDArray[np.float64]
        Phase of first signal in radians.
    phase2 : NDArray[np.float64]
        Phase of second signal in radians.
        
    Returns
    -------
    float
        PLV value in range [0, 1].
        - 0: No phase locking (uniform phase difference)
        - 1: Perfect phase locking (constant phase difference)
        
    Notes
    -----
    This is a simple implementation for educational purposes.
    Full implementation with epoch handling will be in G01.
    
    Examples
    --------
    >>> phase1 = np.angle(hilbert(signal1))
    >>> phase2 = np.angle(hilbert(signal2))
    >>> plv = compute_plv_simple(phase1, phase2)
    """
    phase_diff = phase1 - phase2
    return np.abs(np.mean(np.exp(1j * phase_diff)))


print("✓ compute_plv_simple() function defined")
print("  Note: PLV = resultant_vector_length(phase_difference)!")

In [ ]:
# ============================================================================
# VISUALIZATION 13: PLV and Phase Difference Distribution
# ============================================================================

# Create three scenarios with different PLV levels
np.random.seed(2024)
fs_demo = 250
duration_demo = 3.0
t_demo = np.arange(0, duration_demo, 1/fs_demo)
freq_demo = 10

# Scenario 1: High PLV (constant offset)
sig1_high = np.sin(2 * np.pi * freq_demo * t_demo)
sig2_high = np.sin(2 * np.pi * freq_demo * t_demo + np.pi/4)

# Scenario 2: Medium PLV (slow drift + noise)
phase_drift_med = 0.5 * np.sin(2 * np.pi * 0.3 * t_demo)  # Slow oscillation
sig1_med = np.sin(2 * np.pi * freq_demo * t_demo)
sig2_med = np.sin(2 * np.pi * freq_demo * t_demo + np.pi/4 + phase_drift_med)

# Scenario 3: Low PLV (different frequency)
sig1_low = np.sin(2 * np.pi * freq_demo * t_demo)
sig2_low = np.sin(2 * np.pi * (freq_demo + 0.5) * t_demo)

# Compute PLVs
scenarios = [
    (sig1_high, sig2_high, 'High PLV', PRIMARY_GREEN),
    (sig1_med, sig2_med, 'Medium PLV', ACCENT_GOLD),
    (sig1_low, sig2_low, 'Low PLV', PRIMARY_RED)
]

# Create figure: 3 columns (one per scenario), 2 rows (signal + polar)
fig = plt.figure(figsize=(15, 8))

for i, (s1, s2, label, color) in enumerate(scenarios):
    # Extract phases
    p1 = np.angle(hilbert(s1))
    p2 = np.angle(hilbert(s2))
    phase_diff = compute_phase_difference(p1, p2)
    plv = compute_plv_simple(p1, p2)
    
    # Top row: Phase difference over time
    ax_top = fig.add_subplot(2, 3, i + 1)
    ax_top.plot(t_demo * 1000, phase_diff, color=color, linewidth=1)
    ax_top.set_ylim(-4, 4)
    ax_top.set_xlabel('Time (ms)', fontsize=10)
    ax_top.set_ylabel('Δφ (rad)', fontsize=10)
    ax_top.set_title(f'{label}', fontsize=12, fontweight='bold', color=color)
    ax_top.grid(True, alpha=0.3)
    
    # Bottom row: Polar histogram
    ax_bot = fig.add_subplot(2, 3, i + 4, projection='polar')
    plot_phase_polar_histogram(phase_diff, ax=ax_bot, color=color, 
                               title=f'PLV = {plv:.2f}')
    
    # Add mean vector
    mean_dir = circular_mean(phase_diff)
    ax_bot.annotate('', xy=(mean_dir, ax_bot.get_ylim()[1] * 0.6), xytext=(mean_dir, 0),
                    arrowprops=dict(arrowstyle='->', color='black', lw=2))

plt.suptitle('Visualization 13: PLV Reflects Phase Difference Concentration', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Summary:")
print("  High PLV → Phase difference is constant (concentrated distribution)")
print("  Medium PLV → Phase difference varies but has preferred value")
print("  Low PLV → Phase difference covers all values (uniform distribution)")

---

## 10. Practical Considerations

When working with phase in real EEG/MEG data, several practical issues arise:

1. **Sampling rate**: Need enough samples per cycle for accurate phase estimation
2. **Filter bandwidth**: Too narrow = poor temporal resolution; too wide = phase less meaningful
3. **Noise**: Corrupts phase estimates, especially at low amplitudes
4. **Edge effects**: Phase estimates unreliable at signal boundaries
5. **Low amplitude periods**: Phase is essentially random when amplitude is near zero

A common practice is to **mask phase estimates** when the signal amplitude is below a threshold.

In [ ]:
# ============================================================================
# FUNCTION: Mask Low Amplitude Phase
# ============================================================================

def mask_low_amplitude_phase(phase: NDArray[np.float64],
                              amplitude: NDArray[np.float64],
                              threshold_percentile: float = 10.0) -> NDArray[np.float64]:
    """
    Mask phase estimates where amplitude is below a threshold.
    
    Low amplitude regions have unreliable phase estimates (dominated by noise).
    This function sets phase to NaN where amplitude is in the lowest percentile.
    
    Parameters
    ----------
    phase : NDArray[np.float64]
        Phase values in radians.
    amplitude : NDArray[np.float64]
        Amplitude (envelope) values.
    threshold_percentile : float, optional
        Percentile below which amplitude is considered too low (default: 10).
        
    Returns
    -------
    NDArray[np.float64]
        Phase array with NaN where amplitude is below threshold.
        
    Examples
    --------
    >>> envelope = np.abs(hilbert(filtered_signal))
    >>> phase = np.angle(hilbert(filtered_signal))
    >>> phase_masked = mask_low_amplitude_phase(phase, envelope, threshold_percentile=15)
    """
    threshold = np.percentile(amplitude, threshold_percentile)
    phase_masked = phase.copy()
    phase_masked[amplitude < threshold] = np.nan
    return phase_masked


print("✓ mask_low_amplitude_phase() function defined")

In [ ]:
# ============================================================================
# VISUALIZATION 14: Amplitude Masking of Phase
# ============================================================================

# Create signal with varying amplitude (alpha bursts)
np.random.seed(333)
fs_mask = 250
duration_mask = 3.0
t_mask = np.arange(0, duration_mask, 1/fs_mask)

# Amplitude modulation: bursts of alpha
amp_mod = 0.2 + 0.8 * np.maximum(0, np.sin(2 * np.pi * 0.5 * t_mask))
alpha_bursts = amp_mod * np.sin(2 * np.pi * 10 * t_mask)

# Add noise
noisy_alpha = alpha_bursts + 0.3 * np.random.randn(len(t_mask))

# Filter to alpha band
filtered_alpha = bandpass_filter(noisy_alpha, 8, 12, fs_mask)

# Extract amplitude and phase
analytic_mask = hilbert(filtered_alpha)
amplitude_mask = np.abs(analytic_mask)
phase_mask = np.angle(analytic_mask)

# Create masked phase
phase_masked = mask_low_amplitude_phase(phase_mask, amplitude_mask, threshold_percentile=20)

# Create figure
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# Signal with amplitude modulation
axes[0].plot(t_mask * 1000, noisy_alpha, color='gray', alpha=0.5, label='Noisy signal')
axes[0].plot(t_mask * 1000, filtered_alpha, color=PRIMARY_BLUE, linewidth=1.5, label='Filtered (8-12 Hz)')
axes[0].plot(t_mask * 1000, amplitude_mask, color=PRIMARY_RED, linewidth=2, label='Envelope')
axes[0].plot(t_mask * 1000, -amplitude_mask, color=PRIMARY_RED, linewidth=2)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Alpha Signal with Varying Amplitude (Bursts)', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Amplitude with threshold
threshold_val = np.percentile(amplitude_mask, 20)
axes[1].plot(t_mask * 1000, amplitude_mask, color=PRIMARY_BLUE, linewidth=1.5)
axes[1].axhline(y=threshold_val, color=PRIMARY_RED, linestyle='--', linewidth=2, 
                label=f'20th percentile threshold = {threshold_val:.2f}')
axes[1].fill_between(t_mask * 1000, 0, amplitude_mask, 
                     where=amplitude_mask < threshold_val, alpha=0.3, color=PRIMARY_RED)
axes[1].set_ylabel('Amplitude', fontsize=11)
axes[1].set_title('Amplitude with Low-Value Regions Highlighted', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Raw phase (noisy in low-amplitude regions)
axes[2].plot(t_mask * 1000, phase_mask, color=ACCENT_PURPLE, linewidth=0.8)
axes[2].set_ylabel('Phase (rad)', fontsize=11)
axes[2].set_title('Raw Phase: Noisy During Low-Amplitude Periods', 
                  fontsize=12, fontweight='bold', color=PRIMARY_RED)
axes[2].set_ylim(-4, 4)
axes[2].grid(True, alpha=0.3)

# Masked phase (NaN where amplitude is low)
axes[3].plot(t_mask * 1000, phase_masked, color=PRIMARY_GREEN, linewidth=0.8)
axes[3].set_xlabel('Time (ms)', fontsize=11)
axes[3].set_ylabel('Phase (rad)', fontsize=11)
axes[3].set_title('Masked Phase: NaN Where Amplitude < Threshold (Reliable Estimates Only)', 
                  fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[3].set_ylim(-4, 4)
axes[3].grid(True, alpha=0.3)

plt.suptitle('Visualization 14: Masking Unreliable Phase Estimates', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

n_masked = np.sum(np.isnan(phase_masked))
print(f"Masked {n_masked} samples ({100*n_masked/len(phase_masked):.1f}%) with low amplitude")
print("→ Phase estimates in remaining samples are more reliable!")

---

## 11. Phase Synchronization in Hyperscanning

**Why Phase Matters for Hyperscanning**

When two people interact (speaking, cooperating, making eye contact), their brain rhythms
can become **phase-synchronized**:

- **Mutual synchronization**: Both brains adjust their phase
- **Leader-follower dynamics**: One brain leads, the other follows
- **Shared attention**: Common external stimulus drives both brains

**What to Look For**

| Pattern | Phase Difference | Interpretation |
|---------|-----------------|----------------|
| In-phase sync | ≈ 0 | Brains oscillating together |
| Anti-phase sync | ≈ π | Alternating pattern |
| Consistent lag | Constant non-zero | Possible leader-follower |
| No sync | Random drift | Independent processing |

**Practical Considerations for Hyperscanning**

1. **Cross-frequency coupling**: Person A's alpha phase → Person B's gamma amplitude
2. **Delay estimation**: Neural/motor delays create phase lags
3. **Artifact rejection**: Movement artifacts create spurious synchrony
4. **Sample size**: Brief moments of synchrony need careful statistics

In [ ]:
# ============================================================================
# VISUALIZATION 15: Hyperscanning Simulation - Two "Brains" Synchronizing
# ============================================================================

# Simulate two "brains" with dynamic synchronization
np.random.seed(444)
fs_hyper = 250
duration_hyper = 10.0  # 10 seconds of "interaction"
t_hyper = np.arange(0, duration_hyper, 1/fs_hyper)

# Brain 1: base alpha oscillation
freq_alpha = 10  # Hz
phase_brain1 = 2 * np.pi * freq_alpha * t_hyper

# Brain 2: same frequency but with varying phase relationship
# - First 3 seconds: independent (random phase drift)
# - Seconds 3-7: synchronized (person A and B cooperating)
# - Seconds 7-10: independent again (task ends)

# Create phase coupling modulation
coupling_strength = np.zeros(len(t_hyper))
coupling_strength[(t_hyper >= 3) & (t_hyper < 7)] = 0.95  # Strong coupling during cooperation

# Brain 2 phase: combination of independent and coupled dynamics
phase_drift = 0.5 * np.random.randn(len(t_hyper)).cumsum() / fs_hyper  # Random walk
phase_brain2_base = 2 * np.pi * freq_alpha * t_hyper + phase_drift

# Mix coupled and uncoupled
phase_brain2 = (coupling_strength * phase_brain1 + 
                (1 - coupling_strength) * phase_brain2_base)

# Generate signals
signal_brain1 = np.sin(phase_brain1) + 0.2 * np.random.randn(len(t_hyper))
signal_brain2 = np.sin(phase_brain2) + 0.2 * np.random.randn(len(t_hyper))

# Extract instantaneous phase via Hilbert
analytic1 = hilbert(bandpass_filter(signal_brain1, 8, 12, fs_hyper))
analytic2 = hilbert(bandpass_filter(signal_brain2, 8, 12, fs_hyper))
inst_phase1 = np.angle(analytic1)
inst_phase2 = np.angle(analytic2)

# Compute running PLV (sliding window)
window_samples = int(0.5 * fs_hyper)  # 500ms window
step_samples = int(0.1 * fs_hyper)    # 100ms step
running_plv = []
running_time = []

for start in range(0, len(t_hyper) - window_samples, step_samples):
    end = start + window_samples
    phase_diff = inst_phase1[start:end] - inst_phase2[start:end]
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    running_plv.append(plv)
    running_time.append(t_hyper[start + window_samples // 2])

running_plv = np.array(running_plv)
running_time = np.array(running_time)

# Create figure
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Signals from both "brains"
axes[0].plot(t_hyper, signal_brain1, color=PRIMARY_BLUE, linewidth=0.8, 
             alpha=0.7, label='Brain 1 (alpha)')
axes[0].plot(t_hyper, signal_brain2 + 3, color=PRIMARY_RED, linewidth=0.8, 
             alpha=0.7, label='Brain 2 (alpha)')
axes[0].axvspan(3, 7, color=PRIMARY_GREEN, alpha=0.2, label='Cooperation phase')
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Hyperscanning Simulation: Two Brains During Social Interaction', 
                  fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].set_xlim(0, 10)
axes[0].grid(True, alpha=0.3)

# Zoomed view during synchrony
sync_slice = (t_hyper >= 4.5) & (t_hyper < 5.5)
axes[1].plot(t_hyper[sync_slice] * 1000, signal_brain1[sync_slice], 
             color=PRIMARY_BLUE, linewidth=1.5, label='Brain 1')
axes[1].plot(t_hyper[sync_slice] * 1000, signal_brain2[sync_slice], 
             color=PRIMARY_RED, linewidth=1.5, label='Brain 2')
axes[1].set_ylabel('Amplitude', fontsize=11)
axes[1].set_title('Zoomed: During Cooperation (4.5-5.5s) - Synchronized!', 
                  fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Phase difference over time
phase_diff_all = wrap_phase(inst_phase1 - inst_phase2)
axes[2].scatter(t_hyper, phase_diff_all, c=t_hyper, cmap='viridis', s=1, alpha=0.5)
axes[2].axvspan(3, 7, color=PRIMARY_GREEN, alpha=0.2)
axes[2].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_ylabel('Phase Difference (rad)', fontsize=11)
axes[2].set_title('Phase Difference Over Time (Colored by Time)', 
                  fontsize=12, fontweight='bold')
axes[2].set_ylim(-np.pi, np.pi)
axes[2].set_xlim(0, 10)
axes[2].grid(True, alpha=0.3)

# Running PLV
axes[3].plot(running_time, running_plv, color=ACCENT_PURPLE, linewidth=2)
axes[3].axvspan(3, 7, color=PRIMARY_GREEN, alpha=0.2, label='Cooperation phase')
axes[3].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
axes[3].fill_between(running_time, 0, running_plv, alpha=0.3, color=ACCENT_PURPLE)
axes[3].set_xlabel('Time (s)', fontsize=11)
axes[3].set_ylabel('PLV', fontsize=11)
axes[3].set_title('Running PLV (500ms window) - Captures Synchronization Dynamics', 
                  fontsize=12, fontweight='bold')
axes[3].set_ylim(0, 1)
axes[3].set_xlim(0, 10)
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

# Add annotations
axes[3].annotate('Independent', xy=(1.5, running_plv[15]-0.1), fontsize=10, ha='center',
                 color=PRIMARY_RED, fontweight='bold')
axes[3].annotate('Synchronized!', xy=(5, 0.9), fontsize=10, ha='center',
                 color=PRIMARY_GREEN, fontweight='bold')
axes[3].annotate('Independent', xy=(8.5, 0.25), fontsize=10, ha='center',
                 color=PRIMARY_RED, fontweight='bold')

plt.suptitle('Visualization 15: Phase Synchrony Reveals Social Coupling', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Before cooperation (0-3s): Low PLV, phases drift independently")
print("- During cooperation (3-7s): High PLV, phases lock together")
print("- After cooperation (7-10s): PLV drops, phases become independent again")
print("\n→ Phase synchronization quantifies the 'neural coupling' between interacting brains!")

---

## 12. Hands-On Exercises

### Exercise 1: Linear vs Circular Mean

Generate 100 random phases uniformly distributed on $[-\pi, \pi]$.

1. Compute the **linear mean** (regular average)
2. Compute the **circular mean** using the vector method
3. Visualize both on a polar plot

**Questions:**
- Why is the linear mean misleading for circular data?
- What should the circular mean be for uniform phases?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 1
# ==============================================

# Generate 100 uniform random phases


# Compute linear mean


# Compute circular mean


# Visualize on polar plot



<details>
<summary>💡 Click to see solution</summary>

```python
# Generate 100 uniform random phases
np.random.seed(123)
uniform_phases = np.random.uniform(-np.pi, np.pi, 100)

# Compute linear mean
linear_mean = np.mean(uniform_phases)

# Compute circular mean
mean_x = np.mean(np.cos(uniform_phases))
mean_y = np.mean(np.sin(uniform_phases))
circ_mean = np.arctan2(mean_y, mean_x)
R = np.sqrt(mean_x**2 + mean_y**2)

# Visualize on polar plot
fig, ax = plt.subplots(1, 1, figsize=(8, 8), subplot_kw={'projection': 'polar'})

# Plot phases
ax.scatter(uniform_phases, np.ones(100), color=PRIMARY_BLUE, alpha=0.5, s=50)

# Mark linear mean (misleading!)
ax.scatter(linear_mean, 1.3, color=PRIMARY_RED, s=200, marker='X', 
           label=f'Linear mean = {linear_mean:.2f} rad')

# Mark circular mean
ax.annotate('', xy=(circ_mean, 0.8), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color=PRIMARY_GREEN, lw=3))
ax.scatter(circ_mean, 0.8, color=PRIMARY_GREEN, s=200, marker='o', 
           label=f'Circular mean = {circ_mean:.2f} rad (R={R:.2f})')

ax.set_title(f'Linear vs Circular Mean for Uniform Phases', fontsize=12, fontweight='bold')
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
plt.tight_layout()
plt.show()

print(f"Linear mean: {linear_mean:.3f} rad")
print(f"Circular mean: {circ_mean:.3f} rad")
print(f"Resultant vector length R: {R:.3f}")
print("\n→ For uniform phases, R ≈ 0, meaning the circular mean is ill-defined!")
print("→ The linear mean gives a spurious 'average' that has no meaning.")
```

</details>

---

### Exercise 2: Concentration and Resultant Vector Length

Generate phases from a **von Mises distribution** with different concentration parameters
(κ = 0.5, 2, 10). For each:

1. Generate 200 samples
2. Compute the resultant vector length R
3. Visualize on polar histograms

**Questions:**
- How does R relate to κ?
- What would R be for perfectly synchronized oscillators?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 2
# ==============================================

# Define concentration parameters
kappas = [0.5, 2, 10]

# For each kappa:
# - Generate 200 samples from von Mises distribution
# - Compute R
# - Plot polar histogram



<details>
<summary>💡 Click to see solution</summary>

```python
from scipy.stats import vonmises

kappas = [0.5, 2, 10]
n_samples = 200

fig, axes = plt.subplots(1, 3, figsize=(14, 5), subplot_kw={'projection': 'polar'})

for ax, kappa in zip(axes, kappas):
    # Generate von Mises samples centered at 0
    phases = vonmises.rvs(kappa, size=n_samples)
    
    # Compute R
    R = resultant_vector_length(phases)
    
    # Plot polar histogram
    plot_phase_polar_histogram(phases, n_bins=24, ax=ax)
    
    # Add mean vector
    mean_angle = circular_mean(phases)
    ax.annotate('', xy=(mean_angle, R * 0.8), xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color=PRIMARY_RED, lw=3))
    
    ax.set_title(f'κ = {kappa}\nR = {R:.3f}', fontsize=12, fontweight='bold')

plt.suptitle('Von Mises Concentration vs Resultant Vector Length', 
             fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("As κ increases, phases become more concentrated → R increases")
print("For perfectly synchronized oscillators (identical phases), R = 1")
```

</details>

---

### Exercise 3: Perfect Phase Locking

Create two 10 Hz sinusoids with a **known constant phase offset** of π/4:

1. Generate 2 seconds of each signal at 250 Hz
2. Extract their instantaneous phases using Hilbert transform
3. Compute the PLV

**Questions:**
- What PLV do you expect? Why?
- What happens if you add noise to both signals?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 3
# ==============================================

# Parameters
fs = 250
duration = 2.0
freq = 10  # Hz
phase_offset = np.pi / 4

# Generate two signals with known phase offset


# Extract instantaneous phases (hint: use Hilbert transform)


# Compute PLV


# Try adding noise and recompute



<details>
<summary>💡 Click to see solution</summary>

```python
# Parameters
fs = 250
duration = 2.0
t = np.arange(0, duration, 1/fs)
freq = 10
phase_offset = np.pi / 4

# Generate two signals with known phase offset
signal1 = np.sin(2 * np.pi * freq * t)
signal2 = np.sin(2 * np.pi * freq * t + phase_offset)

# Extract phases using Hilbert transform
phase1 = np.angle(hilbert(signal1))
phase2 = np.angle(hilbert(signal2))

# Compute PLV
plv_clean = compute_plv_simple(phase1, phase2)
print(f"PLV (no noise): {plv_clean:.4f}")
print(f"Expected: 1.0 (constant phase difference = perfect locking)")

# Now add noise
noise_level = 0.5
signal1_noisy = signal1 + noise_level * np.random.randn(len(t))
signal2_noisy = signal2 + noise_level * np.random.randn(len(t))

# Filter to extract 10 Hz component
signal1_filt = bandpass_filter(signal1_noisy, 8, 12, fs)
signal2_filt = bandpass_filter(signal2_noisy, 8, 12, fs)

# Extract phases
phase1_noisy = np.angle(hilbert(signal1_filt))
phase2_noisy = np.angle(hilbert(signal2_filt))

# Compute PLV
plv_noisy = compute_plv_simple(phase1_noisy, phase2_noisy)
print(f"\nPLV (noise level {noise_level}): {plv_noisy:.4f}")
print("Noise reduces PLV because phase estimates become less reliable")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Clean signals
axes[0, 0].plot(t[:100], signal1[:100], label='Signal 1', color=PRIMARY_BLUE)
axes[0, 0].plot(t[:100], signal2[:100], label='Signal 2', color=PRIMARY_RED)
axes[0, 0].set_title(f'Clean Signals - PLV = {plv_clean:.3f}', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Phase difference (clean)
axes[0, 1].hist(wrap_phase(phase1 - phase2), bins=50, color=PRIMARY_GREEN, alpha=0.7)
axes[0, 1].axvline(x=phase_offset, color=PRIMARY_RED, linestyle='--', 
                   label=f'True offset = π/4')
axes[0, 1].set_title('Phase Difference Distribution (Clean)', fontweight='bold')
axes[0, 1].legend()

# Noisy signals
axes[1, 0].plot(t[:100], signal1_filt[:100], label='Signal 1 filtered', color=PRIMARY_BLUE)
axes[1, 0].plot(t[:100], signal2_filt[:100], label='Signal 2 filtered', color=PRIMARY_RED)
axes[1, 0].set_title(f'Noisy Signals (Filtered) - PLV = {plv_noisy:.3f}', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Phase difference (noisy)
axes[1, 1].hist(wrap_phase(phase1_noisy - phase2_noisy), bins=50, color=ACCENT_PURPLE, alpha=0.7)
axes[1, 1].axvline(x=phase_offset, color=PRIMARY_RED, linestyle='--', 
                   label=f'True offset = π/4')
axes[1, 1].set_title('Phase Difference Distribution (Noisy)', fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.show()
```

</details>

---

### Exercise 4: Drifting Phase Difference

Create two sinusoids where the second one has a **slightly different frequency** (10 Hz vs 10.2 Hz):

1. Generate 5 seconds of signals
2. Compute running PLV using 500ms windows
3. Plot the running PLV over time

**Questions:**
- Why does PLV vary over time?
- What is the relationship between frequency difference and PLV?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 4
# ==============================================

# Parameters
fs = 250
duration = 5.0
freq1 = 10.0
freq2 = 10.2  # Slightly different!

# Generate signals


# Compute running PLV with 500ms windows


# Plot running PLV over time



<details>
<summary>💡 Click to see solution</summary>

```python
# Parameters
fs = 250
duration = 5.0
t = np.arange(0, duration, 1/fs)
freq1 = 10.0
freq2 = 10.2

# Generate signals
signal1 = np.sin(2 * np.pi * freq1 * t)
signal2 = np.sin(2 * np.pi * freq2 * t)

# Extract phases
phase1 = np.angle(hilbert(signal1))
phase2 = np.angle(hilbert(signal2))

# Compute running PLV
window_samples = int(0.5 * fs)  # 500ms
step_samples = int(0.05 * fs)   # 50ms step
running_plv = []
running_time = []

for start in range(0, len(t) - window_samples, step_samples):
    end = start + window_samples
    phase_diff = phase1[start:end] - phase2[start:end]
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    running_plv.append(plv)
    running_time.append(t[start + window_samples // 2])

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 9))

# Signals
axes[0].plot(t, signal1, label=f'{freq1} Hz', color=PRIMARY_BLUE, alpha=0.7)
axes[0].plot(t, signal2, label=f'{freq2} Hz', color=PRIMARY_RED, alpha=0.7)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Two Sinusoids with Slightly Different Frequencies', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Phase difference
phase_diff_all = wrap_phase(phase1 - phase2)
axes[1].plot(t, phase_diff_all, color=ACCENT_PURPLE)
axes[1].set_ylabel('Phase Diff (rad)')
axes[1].set_title('Phase Difference Drifts Over Time', fontweight='bold')
axes[1].set_ylim(-np.pi, np.pi)
axes[1].grid(True, alpha=0.3)

# Running PLV
axes[2].plot(running_time, running_plv, color=PRIMARY_GREEN, linewidth=2)
axes[2].fill_between(running_time, 0, running_plv, alpha=0.3, color=PRIMARY_GREEN)
axes[2].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('PLV')
axes[2].set_title('Running PLV - Oscillates as Phases Drift In and Out of Alignment', 
                  fontweight='bold')
axes[2].set_ylim(0, 1)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate full PLV
full_plv = compute_plv_simple(phase1, phase2)
print(f"Overall PLV: {full_plv:.3f}")
print(f"Frequency difference: {freq2 - freq1} Hz")
print(f"Phase completes a full cycle every {1/(freq2-freq1):.1f} seconds")
print("\n→ PLV drops because the phase difference is not constant!")
```

</details>

---

### Exercise 5: Hyperscanning Phase Analysis

Simulate a hyperscanning experiment:
- Two "brains" oscillating at 10 Hz for 8 seconds
- **First half (0-4s)**: Synchronized (cooperating)
- **Second half (4-8s)**: Independent (task ended)

1. Generate the two signals with appropriate phase coupling
2. Compute PLV for each half separately
3. Create a summary visualization

**Questions:**
- How different are the PLV values for each condition?
- What would this look like in a real hyperscanning experiment?

In [ ]:
# ==============================================
# YOUR CODE HERE - Exercise 5
# ==============================================

# Parameters
fs = 250
duration = 8.0
freq = 10

# Generate synchronized phases for first half, independent for second half


# Create signals from phases


# Compute PLV for each half


# Visualize



<details>
<summary>💡 Click to see solution</summary>

```python
np.random.seed(555)

# Parameters
fs = 250
duration = 8.0
t = np.arange(0, duration, 1/fs)
freq = 10
n_samples = len(t)

# Brain 1: consistent 10 Hz oscillation
phase1 = 2 * np.pi * freq * t

# Brain 2: synchronized first half, random drift second half
phase2 = np.zeros(n_samples)

# First half: synchronized (same phase with small jitter)
half = n_samples // 2
phase2[:half] = phase1[:half] + 0.1 * np.random.randn(half).cumsum() / fs

# Second half: independent (random phase walk)
phase2[half:] = phase1[half:] + np.random.uniform(0, 2*np.pi) + \
                np.random.randn(n_samples - half).cumsum() * 0.3 / fs

# Generate signals with some noise
signal1 = np.sin(phase1) + 0.3 * np.random.randn(n_samples)
signal2 = np.sin(phase2) + 0.3 * np.random.randn(n_samples)

# Filter to alpha
signal1_filt = bandpass_filter(signal1, 8, 12, fs)
signal2_filt = bandpass_filter(signal2, 8, 12, fs)

# Extract instantaneous phases
inst_phase1 = np.angle(hilbert(signal1_filt))
inst_phase2 = np.angle(hilbert(signal2_filt))

# Compute PLV for each half
plv_sync = compute_plv_simple(inst_phase1[:half], inst_phase2[:half])
plv_indep = compute_plv_simple(inst_phase1[half:], inst_phase2[half:])

# Create figure
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Signals
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(t, signal1_filt, color=PRIMARY_BLUE, alpha=0.7, label='Brain 1')
ax1.plot(t, signal2_filt + 3, color=PRIMARY_RED, alpha=0.7, label='Brain 2')
ax1.axvline(x=4, color='gray', linestyle='--', linewidth=2)
ax1.axvspan(0, 4, color=PRIMARY_GREEN, alpha=0.1, label='Synchronized')
ax1.axvspan(4, 8, color='gray', alpha=0.1, label='Independent')
ax1.set_ylabel('Amplitude')
ax1.set_title('Hyperscanning Simulation: Two Brains', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Phase difference
ax2 = fig.add_subplot(gs[1, :])
phase_diff = wrap_phase(inst_phase1 - inst_phase2)
ax2.scatter(t, phase_diff, c=t, cmap='RdYlGn_r', s=2, alpha=0.5)
ax2.axvline(x=4, color='gray', linestyle='--', linewidth=2)
ax2.set_ylabel('Phase Diff (rad)')
ax2.set_xlabel('Time (s)')
ax2.set_title('Phase Difference Over Time', fontsize=12, fontweight='bold')
ax2.set_ylim(-np.pi, np.pi)
ax2.grid(True, alpha=0.3)

# Polar plot - Synchronized
ax3 = fig.add_subplot(gs[2, 0], projection='polar')
plot_phase_polar_histogram(phase_diff[:half], n_bins=24, ax=ax3)
ax3.set_title(f'Synchronized\nPLV = {plv_sync:.3f}', fontsize=11, fontweight='bold', 
              color=PRIMARY_GREEN)

# Polar plot - Independent
ax4 = fig.add_subplot(gs[2, 1], projection='polar')
plot_phase_polar_histogram(phase_diff[half:], n_bins=24, ax=ax4)
ax4.set_title(f'Independent\nPLV = {plv_indep:.3f}', fontsize=11, fontweight='bold',
              color=PRIMARY_RED)

# Bar comparison
ax5 = fig.add_subplot(gs[2, 2])
conditions = ['Synchronized\n(0-4s)', 'Independent\n(4-8s)']
plv_values = [plv_sync, plv_indep]
colors = [PRIMARY_GREEN, PRIMARY_RED]
bars = ax5.bar(conditions, plv_values, color=colors, edgecolor='black', linewidth=2)
ax5.set_ylabel('PLV')
ax5.set_ylim(0, 1)
ax5.set_title('PLV Comparison', fontsize=11, fontweight='bold')
ax5.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
for bar, val in zip(bars, plv_values):
    ax5.text(bar.get_x() + bar.get_width()/2, val + 0.03, f'{val:.3f}', 
             ha='center', fontweight='bold')
ax5.grid(True, alpha=0.3, axis='y')

plt.suptitle('Exercise 5: Hyperscanning Phase Analysis', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"PLV during synchronization: {plv_sync:.3f}")
print(f"PLV during independence: {plv_indep:.3f}")
print(f"Difference: {plv_sync - plv_indep:.3f}")
print("\n→ Phase synchronization successfully captures the coupling between brains!")
```

</details>

---

## 13. Summary

### Key Concepts

| Concept | Definition | Key Point |
|---------|------------|-----------|
| **Phase as circular variable** | Phase lives on a circle $[-\pi, \pi]$ | Standard arithmetic doesn't apply! |
| **Wrapping** | Keeps phase in $[-\pi, \pi]$ | Essential after any phase operation |
| **Unwrapping** | Removes artificial $2\pi$ jumps | Reveals continuous phase evolution |
| **Phase difference** | $\Delta\phi = \phi_1 - \phi_2$ wrapped | Measures instantaneous relationship |
| **Circular mean** | Vector average: $\bar{\phi} = \arctan2(\bar{y}, \bar{x})$ | Proper mean for circular data |
| **Resultant vector R** | Length of mean vector, $R \in [0, 1]$ | Measures phase concentration |
| **Circular variance** | $V = 1 - R$ | High when phases are dispersed |
| **PLV** | R of phase differences | Measures phase synchronization |

### Function Reference

```python
# Phase operations
wrapped = wrap_phase(phase)              # Keep in [-π, π]
unwrapped = unwrap_phase(phase)          # Remove discontinuities
diff = compute_phase_difference(p1, p2)  # Wrapped difference

# Circular statistics
mean_angle = circular_mean(phases)       # Vector average
R = resultant_vector_length(phases)      # Concentration [0,1]
V = circular_variance(phases)            # Dispersion = 1-R
s = circular_std(phases)                 # √(-2 ln R)

# Visualization
plot_phase_polar_histogram(phases, n_bins, ax)  # Rose plot
plot_phase_on_circle(phases, ax, show_mean)     # Scatter on circle

# Practical tools
masked = mask_low_amplitude_phase(phase, amp, pct)  # Mask unreliable
plv = compute_plv_simple(phase1, phase2)            # Phase locking
```

### Key Takeaways

1. **Phase is fundamentally circular** - always use circular statistics
2. **R (resultant vector length)** is the core metric for phase concentration
3. **PLV = R of phase differences** - bridges circular stats and connectivity
4. **Low amplitude → unreliable phase** - consider amplitude masking
5. **Same frequency required** for meaningful phase comparison
6. **Hyperscanning** uses phase synchronization to quantify inter-brain coupling

---

## 14. Discussion Questions

1. **Circularity Challenge**: Why would computing the arithmetic mean of phases 
   give misleading results? Can you think of real-world examples beyond neural data 
   where circular statistics are needed?

2. **Unwrapping Limitations**: In what situations would phase unwrapping fail or 
   give incorrect results? How would you detect when unwrapping has gone wrong?

3. **R vs Linear Correlation**: How does the resultant vector length R differ from 
   a linear correlation coefficient? What are the advantages of R for phase data?

4. **Amplitude Threshold**: When masking phases based on amplitude, how would you 
   choose an appropriate threshold? What are the trade-offs between being too 
   strict vs too lenient?

5. **Hyperscanning Interpretation**: If you observe high PLV between two brains, 
   what are the possible explanations beyond "neural coupling"? What controls 
   would you need to rule out confounds?

---

## Next Steps

In the next notebook (**B03: Amplitude Envelope**), we will:
- Explore the amplitude/envelope of oscillations
- Understand amplitude-amplitude coupling
- Learn about amplitude envelope correlation (AEC)
- See how amplitude modulation relates to connectivity

Together with phase (this notebook), amplitude completes the picture of how 
neural oscillations can be characterized and compared across brain regions or 
between interacting individuals.

---

*Notebook completed. All functions will be available in `src/phase.py` for use in subsequent notebooks.*